In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!apt install aria2

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!pip install wandb

In [ ]:
pwd

In [ ]:
!aria2c -x 16 -s 16 https://www.crcv.ucf.edu/data/UCF101/UCF101.rar --check-certificate=false

In [ ]:
!unrar x UCF101.rar

In [ ]:
!pwd

In [ ]:
!pip install decord

In [ ]:
!ls

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as tv
from PIL import Image
import decord
import numpy as np
from einops import reduce
from tqdm import tqdm
import gc
from transformers import Blip2Processor, Blip2Model
from transformers import AutoImageProcessor, Dinov2Model
from datetime import datetime
import logging
import os
import random

random.seed(42)
torch.manual_seed(42)

In [ ]:
class UCFDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, video_dir, subset, video_list_file, frames_per_clip=16):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.video_dir = video_dir
        self.subset = subset
        self.video_list_file = video_list_file
        self.video_list = []
        self.labels = []

        for i in [1, 2, 3]:
            file_path = f'{dataset_dir}/{video_list_file}{i}.txt'
            with open(file_path, 'r') as video_names_file:
                if self.subset == "train":
                    tempvideo_list, templabels = zip(*[line.strip().split() for line in video_names_file])
                    self.video_list.extend(tempvideo_list)
                    self.labels.extend(templabels)
                else:
                    self.video_list.extend([line.strip() for line in video_names_file])
                    self.labels.extend([None] * len(self.video_list))

        self.frames_per_clip = frames_per_clip

        self.transform = tv.transforms.Compose([
            tv.transforms.Resize((224, 224)),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616]),
        ])

    def __len__(self):
        return len(self.video_list)
        

    def __getitem__(self, idx):
        videoname = self.video_list[idx]
        video_path = f'{self.video_dir}/{videoname}'
        vid = decord.VideoReader(video_path, ctx=decord.cpu(0))
        nframes = len(vid)

        if nframes <= self.frames_per_clip:
            idxs = np.arange(self.frames_per_clip) % nframes
        else:
            idxs = np.linspace(0, nframes - 1, self.frames_per_clip).astype(int)

        imgs = [self.transform(Image.fromarray(vid[i].asnumpy())) for i in idxs]
        imgs = torch.stack(imgs, dim=1)

        if self.subset == "train":
            label = int(self.labels[idx]) - 1
        else:
            with open(f'{self.dataset_dir}/classInd.txt', 'r') as class_indices:
                class_map = {line.split()[1]: int(line.split()[0]) for line in class_indices}
            label = class_map[videoname.split('/')[0]]

        return imgs, label

In [ ]:
class MLP(nn.Module):
    def __init__(self, dim, n_class, dino):
        super().__init__()
        self.dino = dino
        self.dino.eval()
        self.mlp = nn.Linear(dim, n_class)
        self.h_dim = dim

    def forward(self, inp):
        with torch.no_grad():
            B, C, T, H, W = inp.shape
            inp = inp.transpose(1, 2).reshape(B * T, C, H, W)
            output = self.dino(inp).last_hidden_state

        output = output[:, 0, :]
        output = output.reshape(B, T, -1)
        output = output.mean(dim=-2)
        return self.mlp(output)

In [2]:
!git clone https://github.com/BhashkarKumar2/anomally_detection

Cloning into 'anomally_detection'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 22 (delta 8), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 105.41 KiB | 3.90 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
dataset_dir = "/kaggle/working/anomally_detection/UCF101TrainTestSplits-RecognitionTask"
video_dir = "/kaggle/working/UCF-101"
frames_per_clip = 8
batch_size = 64
num_workers = 8
pin_memory = True
device = "cuda"
epochs = 10
lr = 2e-5
num_classes = 101

train_val_data = UCFDataset(dataset_dir, video_dir, "train", "trainlist0", frames_per_clip)
train_len = int(0.85 * len(train_val_data))
train_data, val_data = random_split(train_val_data, [train_len, len(train_val_data) - train_len])

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_loader = DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [ ]:
dino = Dinov2Model.from_pretrained('facebook/dinov2-base')
for param in dino.parameters():
    param.requires_grad = False

In [ ]:
model = MLP(768, num_classes, dino)
model.to(device)

for name, params in model.named_parameters():
    if params.requires_grad:
        print(name)

loss_criterion = nn.CrossEntropyLoss()
T_max = epochs * len(train_loader)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=T_max)

In [ ]:
import wandb

wandb.login(key="7d4e6e189c75e2a9a1c2b7ad8dfc76d79ca9b67a")
wandb.init()

In [ ]:
def top_k_accuracy(output, target, topk=(1, 5)):
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
def train_step(loader, epoch, scheduler):
    model.train()
    total_loss = 0
    correct_top1_train = 0
    correct_top5_train = 0
    for batch_id, (video_data, labels) in enumerate(loader):
        video_data, labels = video_data.to(device), labels.to(device)

        optimizer.zero_grad()
        prediction = model(video_data)
        loss = loss_criterion(prediction, labels)
        total_loss += loss.item()

        top1, top5 = top_k_accuracy(prediction, labels)
        correct_top1_train += top1.item()
        correct_top5_train += top5.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()

        del video_data, labels
        gc.collect()

    top1_accuracy = correct_top1_train / len(loader)
    top5_accuracy = correct_top5_train / len(loader)
    loss = total_loss / len(loader)
    

    print(f"[Train Epoch {epoch}] Top-1 Acc: {top1_accuracy:.2f}% | Top-5 Acc: {top5_accuracy:.2f}% | Loss: {loss:.4f}")
    return top1_accuracy, top5_accuracy, loss

In [ ]:
def val_step(loader, epoch):
    model.eval()
    total_loss = 0
    correct_top1_train = 0
    correct_top5_train = 0

    with torch.no_grad():
        for batch_id, (video_data, labels) in enumerate(loader):
            video_data, labels = video_data.to(device), labels.to(device)
            prediction = model(video_data)
            loss = loss_criterion(prediction, labels)
            total_loss += loss.item()

            top1, top5 = top_k_accuracy(prediction, labels)
            correct_top1_train += top1.item()
            correct_top5_train += top5.item()

            # ✅ Log loss at each step
            wandb.log({"val/loss_step": loss.item()})

    top1_accuracy = correct_top1_train / len(loader)
    top5_accuracy = correct_top5_train / len(loader)
    loss = total_loss / len(loader)

    print(f"[Val Epoch {epoch}] Top-1 Acc: {top1_accuracy:.2f}% | Top-5 Acc: {top5_accuracy:.2f}% | Loss: {loss:.4f}")

    # ✅ Log final metrics at the end of the epoch
    wandb.log({
        "val/epoch": epoch,
        "val/loss_epoch": loss,
        "val/top1_acc": top1_accuracy,
        "val/top5_acc": top5_accuracy
    })

    return top1_accuracy, top5_accuracy, loss

In [ ]:

new_acc=float("-inf")
checkpoint_dir="checkpoints_corrected"
os.makedirs(checkpoint_dir,exist_ok=True)
for epoch in range(1, epochs + 1):
    train_top1, train_top5, train_loss = train_step(train_loader, epoch, scheduler)
    val_top1, val_top5, val_loss = val_step(val_loader, epoch)
    if(val_top1>new_acc):
        new_acc=val_top1
        print(f"Saving model at epoch {epoch}")
        torch.save(model.state_dict(), os.path.join(checkpoint_dir,f"model_epoch_{epoch}.pth"))

